# Grow 70M Pythia to 160M Pythia

Model size table:

| Params | n_layers	| d_model |	n_heads | d_head | Hugging Face Checkpoints |
| ------ | -------- | ------- | ------- | ------ | ------------------------ |
|   70M  |    6     |   512   |    8    |   64   | [Standard](https://huggingface.co/EleutherAI/pythia-70m) |
|  410M  |    24    |  1024   |    16   |   64   | [Standard](https://huggingface.co/EleutherAI/pythia-160m) |


Based on the code in https://github.com/allenai/staged-training/blob/main/gpt_pretrain.py.


## Instantiate Pythia 70M

In [1]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer
import torch
import re
import copy
import json
import importlib
import sys

/Users/vibhamasti/Personal/CMU/S24/Capstone/code/LoRA-Instruction-Finetune/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
importlib.reload(sys.modules['transformers'])

<module 'transformers' from '/Users/vibhamasti/Personal/CMU/S24/Capstone/code/LoRA-Instruction-Finetune/venv/lib/python3.11/site-packages/transformers/__init__.py'>

In [3]:
model_70m = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-70m",
  cache_dir="../.cache/pythia-70m",
)

tokenizer_70m = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-70m",
  cache_dir="../.cache/pythia-70m",
)

inputs = tokenizer_70m("Finish the following sentence:\nRaindrops on roses", return_tensors="pt")
tokens = model_70m.generate(**inputs)
tokenizer_70m.decode(tokens[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/Users/vibhamasti/Personal/CMU/S24/Capstone/code/LoRA-Instruction-Finetune/venv/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Finish the following sentence:\nRaindrops on roses\n\nI have a question for you.'

## Use function preserving to grow the model to 410M

In [4]:
# from transformers import GPT2Tokenizer, GPT2Model
# tokenizer_gpt = GPT2Tokenizer.from_pretrained('gpt2')
# model_gpt = GPT2Model.from_pretrained('gpt2')

# for name, module in model_gpt.named_modules():
#     print(name, module)

In [5]:
import importlib

In [6]:
import grow_depth
import grow_width

In [7]:
importlib.reload(grow_depth)

<module 'grow_depth' from '/Users/vibhamasti/Personal/CMU/S24/Capstone/code/LoRA-Instruction-Finetune/src/grow_depth.py'>

### Output of pre-trained 410M model

In [8]:
model_410m = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-410m",
  cache_dir="../.cache/pythia-410m",
)

tokenizer_410m = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-410m",
  cache_dir="../.cache/pythia-410m",
)

inputs = tokenizer_410m("Finish the following sentence:\nRaindrops on roses", return_tensors="pt")
tokens = model_410m.generate(**inputs)
tokenizer_410m.decode(tokens[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/Users/vibhamasti/Personal/CMU/S24/Capstone/code/LoRA-Instruction-Finetune/venv/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Finish the following sentence:\nRaindrops on roses,\n\nA:\n\nI think'

In [9]:
model_70m_deep = grow_depth.expand_layers(model_70m, 6, 12, expand_type='alternate')
model_70m_deep = grow_depth.expand_layers(model_70m_deep, 12, 24, expand_type='alternate')

In [10]:
inputs = tokenizer_70m("Finish the following sentence:\nRaindrops on roses", return_tensors="pt")
tokens = model_70m_deep.generate(**inputs)
tokenizer_70m.decode(tokens[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Finish the following sentence:\nRaindrops on roses\n\nI have a question for you.'

In [11]:
model_70m_wide = grow_width.expand_width(model_70m_deep, 512, 1024)

In [12]:
inputs = tokenizer_70m("Finish the following sentence:\nRaindrops on roses", return_tensors="pt")
tokens = model_70m_wide.generate(**inputs)
tokenizer_70m.decode(tokens[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Finish the following sentence:\nRaindrops on roses\n\nI have a question for you about'

In [13]:
model_70m_wide

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [14]:
model_70m_wide.config

GPTNeoXConfig {
  "_name_or_path": "EleutherAI/pythia-70m-expand-width-1024",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

In [15]:
model_410m

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [16]:
model_410m.config

GPTNeoXConfig {
  "_name_or_path": "EleutherAI/pythia-410m",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

## 410M to 1.4B

In [17]:
model_1_4b = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-1.4b",
  cache_dir="../.cache/pythia-1.4b",
)

In [18]:
inputs = tokenizer_70m("Finish the following sentence:\nRaindrops on roses", return_tensors="pt")
tokens = model_1_4b.generate(**inputs)
tokenizer_70m.decode(tokens[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Finish the following sentence:\nRaindrops on roses,\n\nI love you.\n\n'

In [19]:
importlib.reload(grow_width)

<module 'grow_width' from '/Users/vibhamasti/Personal/CMU/S24/Capstone/code/LoRA-Instruction-Finetune/src/grow_width.py'>

In [20]:
model_410m_grown = grow_width.expand_width(model_410m, 1024, 2048, attn_heads=16)

In [21]:
inputs = tokenizer_70m("Finish the following sentence:\nRaindrops on roses", return_tensors="pt")
tokens = model_410m_grown.generate(**inputs)
tokenizer_70m.decode(tokens[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Finish the following sentence:\nRaindrops on roses,\n\nA:\n\nI think'

In [22]:
model_410m_grown.config

GPTNeoXConfig {
  "_name_or_path": "EleutherAI/pythia-410m-expand-width-2048",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

In [10]:
# model_70m.embed_out